In [1]:
# ライブラリ
import numpy as np
import pickle as pkl
import pandas as pd 
import os
import warnings
import logging
import configparser
import datetime as dt
import torch
from torch.utils.data import TensorDataset, DataLoader

import sys

from model.multi_scale_ori import *

In [2]:
config = configparser.ConfigParser()
config.read(os.path.join('config.ini'))

['config.ini']

In [3]:
batch_size= int(config['PARAMETER']['infer_batch_size'])
data_path = config['PATH']['infer_data']
model_param_path = config['PATH']['infer_model_param']
output_path = os.path.join('..', '..','output', 'infer_output')

In [4]:
data_path

'/home/ubuntu/accident_detection/data/infer_data/test_X.pkl'

In [5]:
torch.cuda.empty_cache()
n_class = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
msresnet = MSResNet(input_channel=3, layers=[1,1,1,1], num_classes=n_class)
msresnet.to(device)

MSResNet(
  (conv1): Conv1d(3, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer3x3_1): Sequential(
    (0): BasicBlock3x3(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (layer3x3_2): Sequential(
    (0): BasicB

In [6]:
device

device(type='cuda', index=0)

In [7]:
data_name = data_path.split('/')[-1]
model_param_name = model_param_path.split('/')[-1]

inputs_data = pd.read_pickle(data_path)
inputs_dataset = TensorDataset(torch.from_numpy(inputs_data))
inputs_dataloader = torch.utils.data.DataLoader(inputs_dataset,batch_size=batch_size)

In [8]:
inputs_data = inputs_data[[0,1]]
inputs_dataset = TensorDataset(torch.from_numpy(inputs_data))
inputs_dataloader = torch.utils.data.DataLoader(inputs_dataset,batch_size=batch_size)

In [9]:
inputs_data.shape

(2, 750, 3)

In [10]:
len(inputs_data.shape)

3

In [11]:
msresnet.eval()
my_inputsiter = iter(inputs_dataloader)
length = len(inputs_dataloader)
output_list = list()
for i in range(length):
    inputs = next(my_inputsiter)
    inputs = inputs[0]
    inputs = inputs.float().cuda()
    outputs, _ = msresnet(inputs.permute(0, 2, 1))
    outputs = torch.sigmoid(outputs)
    tag_one = torch.ones_like(outputs)
    tag_zero = torch.zeros_like(outputs)
    preds = torch.where(outputs >= 0.5, tag_one, tag_zero)
    preds = preds.data.cpu().numpy()
    preds_expr = preds.tolist()
    output_list.append(preds_expr)
#logger.info("result: {}".format(output_list))
np_output_list = np.array(output_list)

In [12]:
np_output_list.shape

(2, 1)